In [1]:
!pip install --quiet wikipedia==1.4.0 langchain-core==0.3.59

In [2]:
from typing import Annotated
import wikipedia
from langchain_core.tools import tool

In [3]:
@tool
def wikipedia_tool(
    query: Annotated[str, "The Wikipedia search to execute to find key summary information."],
):
    """Use this to search Wikipedia for factual information."""
    try:
        # Step 1: Search using query
        results = wikipedia.search(query)
        
        if not results:
            return "No results found on Wikipedia."
        
        # Step 2: Retrieve page title
        title = results[0]

        # Step 3: Fetch summary
        summary = wikipedia.summary(title, sentences=8, auto_suggest=False, redirect=True)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Successfully executed:\nWikipedia summary: {summary}"

In [4]:
# Test with Apple query
company_name = "Apple Inc."
wiki_summary = wikipedia_tool.invoke(f"{company_name}")
print(wiki_summary)

Successfully executed:
Wikipedia summary: Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley. It is best known for its consumer electronics, software, and services. Founded in 1976 as Apple Computer Company by Steve Jobs, Steve Wozniak and Ronald Wayne, the company was incorporated by Jobs and Wozniak as Apple Computer, Inc. the following year. It was renamed Apple Inc. in 2007 as the company had expanded its focus from computers to consumer electronics. Apple is considered part of the Big Tech group, alongside Nvidia, Microsoft, Alphabet, Amazon, and Meta.


In [5]:
!pip install --quiet langchain-core==0.3.59

In [6]:
import os
from typing import Annotated
import pandas as pd
from langchain_core.tools import tool

In [7]:
@tool
def stock_data_tool(
    company_ticker: Annotated[str, "The ticker symbol of the company to retrieve their stock performance data."], 
    num_days: Annotated[int, "The number of days of stock data required to respond to the user query."]
) -> str:
    """
    Use this to look-up stock performance data for companies to retrieve a table from a CSV. You may need to convert company names into ticker symbols to call this function, e.g, Apple Inc. -> AAPL, and you may need to convert weeks, months, and years, into days.
    """
    
    # Load the CSV for the company requested
    file_path = f"data/{company_ticker}.csv"

    if os.path.exists(file_path) is False:
        return f"Sorry, but data for company {company_ticker} is not available. Please try Apple, Amazon, Meta, Microsoft, Netflix, or Tesla."
    
    stock_df = pd.read_csv(file_path, index_col='Date', parse_dates=True)

    # Ensure the index is in date format
    stock_df.index = stock_df.index.date
    
    # Maximum num_days supported by the dataset
    max_num_days = (stock_df.index.max() - stock_df.index.min()).days
    
    if num_days > max_num_days:
        return "Sorry, but this time period exceeds the data available. Please reduce it to continue."
    
    # Get the most recent date in the DataFrame
    final_date = stock_df.index.max()

    # Filter the DataFrame to get the last num_days of stock data
    filtered_df = stock_df[stock_df.index > (final_date - pd.Timedelta(days=num_days))]

    return f"Successfully executed the stock performance data retrieval tool to retrieve the last *{num_days} days* of data for company **{company_ticker}**:\n\n{filtered_df.to_markdown()}"

In [9]:
retrieved_data = stock_data_tool.invoke({"company_ticker": "META", "num_days": 4})
print(retrieved_data)

Successfully executed the stock performance data retrieval tool to retrieve the last *4 days* of data for company **META**:

|            | Close/Last   |   Volume | Open    | High    | Low       |
|:-----------|:-------------|---------:|:--------|:--------|:----------|
| 2025-05-28 | $643.58      |  9042874 | $642.60 | $650.88 | $642.5472 |
| 2025-05-27 | $642.32      |  9508367 | $635.41 | $643.08 | $632.75   |


In [10]:
!pip install --quiet langchain-core==0.3.59 langchain-experimental==0.3.4

In [11]:
from langchain_core.tools import tool
from typing import Annotated
from langchain_experimental.utilities import PythonREPL

In [12]:
repl = PythonREPL()

In [13]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user. The chart should be displayed using `plt.show()`."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Successfully executed the Python REPL tool.\n\nPython code executed:\n\`\`\`python\n{code}\n\`\`\`\n\nCode output:\n\`\`\`\n{result}\`\`\`"

<>:11: SyntaxWarning: invalid escape sequence '\`'
<>:11: SyntaxWarning: invalid escape sequence '\`'
<>:11: SyntaxWarning: invalid escape sequence '\`'
<>:11: SyntaxWarning: invalid escape sequence '\`'
<>:11: SyntaxWarning: invalid escape sequence '\`'
<>:11: SyntaxWarning: invalid escape sequence '\`'
C:\Users\alber\AppData\Local\Temp\ipykernel_2548\2913511864.py:11: SyntaxWarning: invalid escape sequence '\`'
  return f"Successfully executed the Python REPL tool.\n\nPython code executed:\n\`\`\`python\n{code}\n\`\`\`\n\nCode output:\n\`\`\`\n{result}\`\`\`"
C:\Users\alber\AppData\Local\Temp\ipykernel_2548\2913511864.py:11: SyntaxWarning: invalid escape sequence '\`'
  return f"Successfully executed the Python REPL tool.\n\nPython code executed:\n\`\`\`python\n{code}\n\`\`\`\n\nCode output:\n\`\`\`\n{result}\`\`\`"
C:\Users\alber\AppData\Local\Temp\ipykernel_2548\2913511864.py:11: SyntaxWarning: invalid escape sequence '\`'
  return f"Successfully executed the Python REPL tool.\n\nP

In [14]:
code = f"""
import numpy as np

arr = np.arange(0, 9)
print(arr)
print(2 * arr)
"""

print(python_repl_tool.invoke({"code": code}))

Python REPL can execute arbitrary code. Use with caution.


Successfully executed the Python REPL tool.

Python code executed:
\`\`\`python

import numpy as np

arr = np.arange(0, 9)
print(arr)
print(2 * arr)

\`\`\`

Code output:
\`\`\`
[0 1 2 3 4 5 6 7 8]
[ 0  2  4  6  8 10 12 14 16]
\`\`\`
